In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Study on the Football Statistics and the Correctness of the Odds
### by Kalin Primov

## Abstract
.......

## Introduction
The goal of this study is to see to what extend the odds, which bookmakers assign to games can be used in predicting the final outcome. The data with the odds is taken from *oddsportal* and the data with the games is taken from various sources. Also would be measured the accuracy of odds such as both teams to score, more or less than 2.5 goals and both team to or not to score. The data is for the five major leagues - La Liga, Premier League, Bundesliga, Ligue 1 and Seria A and we will see where and how the predictions and reality are different for the leagues.

## Data Manipulation

### Data Collection
The data is divided into 2 datasets in the '''ginf.csv''' is the data from the _odsportal_ with the odds for the games. The columns in the dataset are:
* id_odsp - unique game identifier
* link_odsp - link to oddsportal page
* adv_stats - boolean if the game has detailed event data
* data
* league
* season
* ht - home team
* at - away team
* fthg - full time home goals
* ftag - full time away goals
* odd_h - home win odds
* odd_d - draw odds
* odd_a - away win odds
* odd_over - over 2.5 goals odds
* odd_under - under 2.5 goals odds
* odd_bts - both teams to score odds
* odd_bts_n - both teams not to score odds
In the ```events.csv``` is the data from the events. We will use it to compare it with the data from ```ginf.csv``` and see if there are other factor, which affect games. The columns in the dataset are:
* id_odsp - unique game identifier
* id_event - unique identifier of event (id_odsp + sort_order)
* sort_order - chronological sequence of events in a game
* time - minutes of the game
* text - text commentary
* event_type - primary event. 11 unique events (1-Attempt(shot), 2-Corner, 3-Foul, 4-Yellow Card, 5-Second yellow card, 6-(Straight) red card, 7-Substitution, 8-Free kick won, 9-Offside, 10-Hand Ball, 11-Penalty conceded)
* event_type2 - secondary event. 4 unique events (12 - Key Pass, 13 - Failed through ball, 14-Sending off, 15-Own goal)
* side - 1-Home, 2-Away
* event_team - team that produced the event
* opponent - team that the event happened against
* player - name of the player involved in the main event
* player2 - name of player involved in secondary event
* player_in - substitude in
* player_out - substitude out
* shot_place - 13 possible placements of the shot
* shot_outcome (1-On target, 2-Off target, 3-Blocked, 4-Hit the post)
* is_goal
* location - location on the pitch
* bodypart - (1- right foot, 2-left foot, 3-head)
* assist_method
* situation - 1-Open Play, 2-Set piece (excluding Direct Free kicks), 3-Corner, 4-Free kick
* fast_break

In [3]:
odds_data = pd.read_csv('data/ginf.csv')
print(odds_data.shape)
events_data = pd.read_csv('d')

(10112, 18)